### Sentinel pipeline

In [1]:
# Packages
# !pip install landsatxplore
import os
import datetime
import gc
import glob
import geopandas
import random
from sentinelsat import SentinelAPI, geojson_to_wkt, read_geojson
import json
from landsatxplore.api import API

In [ ]:
class sentinel1_download_preprocess():
    def __init__(self,username,password,date_1,date_2,lat,lon,query_style,footprint,download):
        try:
            self.username = username
            self.password = password
            self.date_start = datetime.datetime.strptime(date_1, "%d/%m/%y")
            self.date_end = datetime.datetime.strptime(date_2, "%d/%m/%y")
            self.lat = lat
            self.lon = lon
            self.query_style = query_style
            self.footprint = geojson_to_wkt(read_geojson(footprint))
            self.download = download# you want to download
                # configurations
            self.api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
            self.platformname ='Sentinel-2'
            self.orbitdirection = 'DESCENDING'  
            self.processinglevel = 'Level-2A'
            self.cloudcoverpercentage = (0,20)
        except:
            print('Invalid input')
    
    @classmethod
    def parameters(cls):#fucntions of input required parameters
        username = input(' Enter username:')
        password = input('Enter password:')
        date_1 = input('Startdate yyyy/mm/dd:') #start_date
        date_2 = input('Enddate yyyy/mm/dd:') #end_date
        query_style = input('query_style:') #footprint or coordinates
        
        lat = None
        lon = None
        footprint = None
        if query_style == 'coordinate':
            lat = input('latitude:')
            lon = input('longitude:')
        elif query_style == 'footprint':
            footprint = input('"Enter path_geoson_file:') #path where geojson file is stored,
        else:
            assert "Define query attribute"          

        download = input('download:') #True or False
            
        return cls(username, password, date_1, date_2, query_style, footprint, lat, lon, download)


    def sentinel1_download(self):
        global download_candidate
        if self.query_style == 'coordinate':
            download_candidate = self.api.query('POINT({0} {1})'.format(self.lon, self.lat),
                                                date=(self.date_start, self.date_end),
                                                platformname=self.platformname,
                                                orbitdirection=self.orbitdirection,
                                                processinglevel=self.processinglevel,
                                                cloudcoverpercentage=self.cloudcoverpercentage)
        elif self.query_style == 'footprint':
            download_candidate = self.api.query(self.footprint,
                                                 date=(self.date_start, self.date_end),
                                                platformname=self.platformname,
                                                orbitdirection=self.orbitdirection,
                                                processinglevel=self.processinglevel,
                                                cloudcoverpercentage=self.cloudcoverpercentage)


            
            dataproduct = self.api.to_geodataframe(download_candidate)#to put into dataframe
            d = []
            for i in range(len(dataproduct.uuid)):
                product_id = self.api.get_product_odata(dataproduct['uuid'][i])
               # print(f"show this {product_id}")
                for j in product_id.keys():
                    if (j == 'Online' and product_id[j] == True):#check id which is online
                        d.append(dataproduct['uuid'][i])
                    
            if len(d)==0:
                print('No data avail')

            print(f"len of the list = {len(d)}")
            self.api.download(random.choice(d))#randomly download
        else:
            print("Define query attribute")

In [ ]:
user = sentinel1_download_preprocess.from_input()
user.sentinel1_download()

###  Landsat pipeline

In [ ]:
def download_landsat():
    #credentials for authentication
    username = input('username:')
    password = input('password:')
    date_1 = input('date_1:') #start_date
    date_2 = input('date_2:') #end_date
    lat = input('lat:') 
    lon = input('lon:')
    cloud = input('cloud:') #footprint or coordinates
    dataset = input('dataset:')

    username = username
    password = password
    api = API(username, password)
    dataset=  dataset
    latitude= lat                                           
    longitude= lon                                             
    start_date= datetime.datetime.strptime(date_1, "%Y-%m-%d")               #'1995-01-01',
    end_date=datetime.datetime.strptime(date_2, "%Y-%m-%d")        
    max_cloud_cover=  cloud                                       
    
    print(f"{len(scenes)} scenes found.")

    # Process the result
    for scene in scenes:
        print(scene['acquisition_date'].strftime('%Y-%m-%d'))
        # Write scene footprints to disk
        fname = f"{scene['landsat_product_id']}.geojson"
        with open(fname, "w") as f:
            json.dump(scene['spatial_coverage'].__geo_interface__, f)

    api.logout()
    
    ee = EarthExplorer(username, password)

    ee.download('LT51960471995178MPS00', output_dir='./data')

    ee.logout()

In [ ]:
download_landsat()